# 03 Multi-Factor Regression (Macro)

Multi-factor GDP growth regression; weights and VIF.


## Table of Contents
- [Choose features](#choose-features)
- [Fit model](#fit-model)
- [VIF + stability](#vif-stability)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Regression is the bridge between statistics and ML. You will learn:
- single-factor vs multi-factor interpretation,
- robust standard errors,
- coefficient stability and multicollinearity.


## Prerequisites (Quick Self-Check)
- Completed Parts 00–01 (foundations + data).
- Basic algebra comfort (reading coefficient tables, units).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating coefficients as causal without a causal design.
- Ignoring multicollinearity (unstable coefficients).

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/02_regression/03_multifactor_regression_macro.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/02_regression/03_multifactor_regression_macro.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Fit a multi-factor GDP growth regression and learn how to interpret feature weights *carefully*.

This notebook is where multicollinearity becomes real:
- many macro indicators move together
- coefficients can change sign or become unstable when features are correlated



## Primer: pandas time series essentials (indexing, resampling, lags)

Most “mysterious bugs” in time series work come from index and alignment mistakes. This primer gives you the minimum patterns to avoid them.

### 1) DatetimeIndex (the first thing to verify)

Most time-series operations assume a `DatetimeIndex`:

```python
import pandas as pd

df = df.copy()
df.index = pd.to_datetime(df.index)
df = df.sort_index()
assert isinstance(df.index, pd.DatetimeIndex)
```

**Expected output / sanity checks**
- `df.index.min(), df.index.max()` look reasonable
- `df.index.is_monotonic_increasing` is `True`

### 2) Resampling (frequency alignment)

Resampling converts one frequency to another. Choose the aggregation rule intentionally.

```python
# month-end last value (end-of-period)
df_me_last = df.resample("ME").last()

# month-end mean (average-of-period)
df_me_mean = df.resample("ME").mean()

# quarter-end mean
df_q_mean = df.resample("QE").mean()
```

**Interpretation matters**
- `.last()` treats end-of-period value as “the period’s value.”
- `.mean()` treats the period average as “the period’s value.”

### 3) Alignment and merging

When joining series, always check missingness after the join:

```python
merged = df1.join(df2, how="outer").sort_index()
print(merged.isna().sum().sort_values(ascending=False).head(10))
```

### 4) Lags and rolling windows (watch for leakage!)

```python
# lag 1 period (past-only)
df["x_lag1"] = df["x"].shift(1)

# rolling mean using past values ending at t
df["x_roll12"] = df["x"].rolling(12).mean()
```

**Leakage pitfalls**
- `shift(-1)` uses the future.
- `rolling(..., center=True)` uses the future.

### 5) A quick workflow you should repeat

1) Set and verify DatetimeIndex.
2) Resample intentionally (mean vs last).
3) Join and inspect missingness.
4) Add lags/rolls (past-only).
5) `dropna()` to build a clean modeling table.


## Primer: `statsmodels` vs `scikit-learn` (inference vs prediction)

This repo uses both libraries because they serve different goals:

- **Prediction (ML):** optimize out-of-sample accuracy → `scikit-learn`
- **Inference (econometrics):** interpret coefficients + quantify uncertainty → `statsmodels`

### Minimal `statsmodels` OLS pattern

```python
import statsmodels.api as sm

# X: DataFrame of features, y: Series target
Xc = sm.add_constant(X, has_constant="add")  # add intercept
res = sm.OLS(y, Xc).fit()
print(res.summary())
```

**Expected output / sanity check**
- a table with `coef`, `std err`, `t`, `P>|t|`, and a CI column
- coefficient names match your column names

### What you are looking at in `res.summary()`

- **coef**: $\\hat\\beta$ (estimated effect in the model)
- **std err**: estimated uncertainty $\\widehat{SE}(\\hat\\beta)$
- **t**: $\\hat\\beta / \\widehat{SE}(\\hat\\beta)$
- **P>|t|**: p-value for $H_0: \\beta=0$ (conditional on assumptions)
- **[0.025, 0.975]**: 95% confidence interval

### Robust standard errors (change uncertainty, not coefficients)

```python
# Cross-section heteroskedasticity
res_hc3 = res.get_robustcov_results(cov_type="HC3")

# Time series autocorrelation + heteroskedasticity
res_hac = res.get_robustcov_results(cov_type="HAC", cov_kwds={"maxlags": 4})
```

### Common pitfalls (and quick fixes)

- **Forgetting the intercept**
  - Fix: always `add_constant`.
- **Wrong SE for time series**
  - Fix: use HAC when residuals are autocorrelated.
- **Treating p-values as causal proof**
  - Fix: write the identification assumption; otherwise interpret as association.
- **Mixing prediction and inference**
  - Fix: use `sklearn` pipelines + time splits for prediction; use `statsmodels` for coefficient uncertainty.


## Primer: Hypothesis testing (p-values, t-stats, confidence intervals)

You will see p-values, t-statistics, and confidence intervals in regression output (especially `statsmodels`). This primer gives you the minimum to interpret them correctly.

### The objects (plain language)

- **Null hypothesis** $H_0$: the default claim (often “no effect”).
- **Alternative** $H_1$: the claim you consider if the data looks inconsistent with $H_0$.
- **Test statistic**: “how far” your estimate is from the null, in uncertainty units.
- **p-value**: probability (under the null *and model assumptions*) of seeing a test statistic at least as extreme as observed.
- **Confidence interval (CI)**: a range of parameter values consistent with the data under assumptions.

### What a p-value is NOT

- Not the probability $H_0$ is true.
- Not the probability the model is correct.
- Not a measure of economic importance.

### Regression t-test intuition

In OLS, a common test is $H_0: \\beta_j = 0$.

$$
t_j = \\frac{\\hat\\beta_j}{\\widehat{SE}(\\hat\\beta_j)}
$$

If you change your SE estimator (HC3/HAC/cluster), you change $\\widehat{SE}$ and therefore the p-value, even if the coefficient stays the same.

### Expected output / what you should look at in `res.summary()`

- `coef`: effect size (in model units)
- `std err`: uncertainty
- CI columns: magnitude + uncertainty together

### Common pitfalls in this project

- Macro time series often have autocorrelation → naive SE too small → use HAC when interpreting p-values.
- Multiple testing/spec-search can produce small p-values by chance.
- Predictive success ≠ causal interpretation.

### Tiny demo (toy; not project data)

```python
import numpy as np
import pandas as pd
import statsmodels.api as sm

rng = np.random.default_rng(0)
n = 300
x = rng.normal(size=n)
y = 1.0 + 0.5 * x + rng.normal(scale=1.0, size=n)

df = pd.DataFrame({"y": y, "x": x})
X = sm.add_constant(df[["x"]])
res = sm.OLS(df["y"], X).fit()
print(res.summary())
```


<a id="choose-features"></a>
## Choose features

### Goal
Pick a feature set to predict GDP growth.

Recommendations:
- Start small (3-6 predictors) before you go wide.
- Prefer lagged predictors (information available before the quarter).
- Keep a record of your feature list.



### Your Turn (1): Load macro data


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'macro_quarterly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

df.head()



### Your Turn (2): Choose a target + feature list


In [ ]:
# Target
y_col = 'gdp_growth_qoq'

# TODO: Choose features.
# Start with lagged predictors to reduce timing ambiguity.
x_cols = [
    'T10Y2Y_lag1',
    'UNRATE_lag1',
    'FEDFUNDS_lag1',
    # TODO: add 1-3 more
]

df_m = df[[y_col] + x_cols].dropna().copy()
df_m.tail()



### Checkpoint (feature table)


In [ ]:
assert df_m.index.is_monotonic_increasing
assert not df_m.isna().any().any()
assert df_m.shape[0] > 30
...



<a id="fit-model"></a>
## Fit model

### Goal
Fit a multi-factor regression and compare:
- raw coefficients (units matter)
- standardized coefficients (compare relative importance)



### Your Turn (1): Time split and fit OLS


In [ ]:
import statsmodels.api as sm
from src.evaluation import time_train_test_split_index, regression_metrics

split = time_train_test_split_index(len(df_m), test_size=0.2)
train = df_m.iloc[split.train_slice]
test = df_m.iloc[split.test_slice]

X_tr = sm.add_constant(train[x_cols], has_constant='add')
y_tr = train[y_col]
X_te = sm.add_constant(test[x_cols], has_constant='add')
y_te = test[y_col]

res = sm.OLS(y_tr, X_tr).fit()
y_hat = res.predict(X_te)

regression_metrics(y_te.to_numpy(), y_hat.to_numpy())



### Your Turn (2): Standardize predictors and compare standardized coefficients


In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardize X (train-fitted scaler!)
sc = StandardScaler().fit(train[x_cols])
X_tr_s = sc.transform(train[x_cols])
X_te_s = sc.transform(test[x_cols])

# Refit on standardized features
X_tr_s = sm.add_constant(X_tr_s, has_constant='add')
X_te_s = sm.add_constant(X_te_s, has_constant='add')
res_s = sm.OLS(y_tr, X_tr_s).fit()

# TODO: Map standardized coefficients back to feature names (excluding intercept)
...



<a id="vif-stability"></a>
## VIF + stability

### Goal
Measure multicollinearity and see whether coefficients are stable.

Two simple stability checks:
- VIF (collinearity)
- fit on different eras and compare coefficients



### Your Turn (1): VIF table


In [ ]:
from src.econometrics import vif_table

# TODO: Compute VIF on the full feature matrix (no intercept).
vif = vif_table(df_m, x_cols)
vif



### Your Turn (2): Era split coefficient stability


In [ ]:
import pandas as pd
import statsmodels.api as sm

# TODO: Fit the same model on an early era vs a late era and compare coefficients.
mid = int(len(df_m) * 0.5)
early = df_m.iloc[:mid]
late = df_m.iloc[mid:]

res_early = sm.OLS(early[y_col], sm.add_constant(early[x_cols], has_constant='add')).fit()
res_late = sm.OLS(late[y_col], sm.add_constant(late[x_cols], has_constant='add')).fit()

comp = pd.DataFrame({'early': res_early.params, 'late': res_late.params})
comp



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Choose features</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_multifactor_regression_macro — Choose features
import pandas as pd
from src import econometrics

df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)
x_cols = ['T10Y2Y', 'UNRATE', 'FEDFUNDS']
res = econometrics.fit_ols_hac(df, y_col='gdp_growth_qoq', x_cols=x_cols, maxlags=2)
print(res.summary())
```

</details>

<details><summary>Solution: Fit model</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_multifactor_regression_macro — Fit model
# See above.
```

</details>

<details><summary>Solution: VIF + stability</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_multifactor_regression_macro — VIF + stability
from src.econometrics import vif_table
print(vif_table(df.dropna(), ['T10Y2Y', 'UNRATE', 'FEDFUNDS']))
```

</details>

